In [1]:
%cd
%cd desktop
import pandas as pd
import json


/Users/mehulgupta/Desktop


In [32]:
#loading results of YOLOv3 Detection
with open('result_plates.json') as file:
    data = json.load(file)
df = pd.DataFrame(data)
df

FileNotFoundError: [Errno 2] No such file or directory: 'result_plates.json'

In [33]:
med=[]
def check(x):
    for y in x:
        if y['name']=="NumberPlate":
                    return True
    return False
df['plates']=df['objects'].transform(lambda temp:check(temp))

/Users/mehulgupta/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [34]:
df=df[df['plates']]

In [35]:
df['filename']=df['filename'].transform(lambda f:f.split('/')[-1])
in_book=list(df['filename'])

In [36]:
%cd 
%cd desktop/img2
import subprocess
proc=subprocess.Popen('ls', shell=True, stdout=subprocess.PIPE, )
output=proc.communicate()[0]
output=output.decode('utf-8').split('\n')
output=list(output)

/Users/mehulgupta
/Users/mehulgupta/Desktop/img2


In [37]:
p=set(output)-set(in_book)

In [38]:
for x in p:
    %rm $x

usage: rm [-f | -i] [-dPRrvW] file ...
       unlink file


In [40]:
med=[]
def check2(x):
    for y in x:
        if y['name']=="NumberPlate":
                    return y
    return False
df['plate_data']=df['objects'].transform(lambda temp:check2(temp))

In [42]:
df

,frame_id,filename,objects,plates,plate_data
0,1,0.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
1,2,1.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
2,3,10.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
3,4,100.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
5,6,102.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
6,7,103.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
7,8,104.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
8,9,105.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
9,10,106.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
10,11,107.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."


In [44]:
df=df.drop(['frame_id','objects','plates'],axis=1)


In [47]:
df['plate_data'][0]

{'class_id': 0,
 'name': 'NumberPlate',
 'relative_coordinates': {'center_x': 0.49581,
  'center_y': 0.373569,
  'width': 0.090576,
  'height': 0.024831},
 'confidence': 0.474979}

In [49]:
data=df

In [50]:
def details(x,flag_2):
        co=x['relative_coordinates']
        wid=float(co['width'])/2.0
        hi=float(co['height'])/2.0
        if flag_2=='y2':
            return float(co['center_y'])+hi
        if flag_2=='x2':
            return float(co['center_x'])+wid
        if flag_2=='x1':
            return float(co['center_x'])-wid
        if flag_2=='y1':
            return float(co['center_y'])-hi
data['x1']=data['plate_data'].transform(lambda f:details(f,'x1'))    
data['y1']=data['plate_data'].transform(lambda f:details(f,'y1'))  
data['x2']=data['plate_data'].transform(lambda f:details(f,'x2'))  
data['y2']=data['plate_data'].transform(lambda f:details(f,'y2'))  

In [56]:
from PIL import Image
count=0
for x,y in data.iterrows():
        img = Image.open('/Users/mehulgupta/Desktop/img2/'+str(y['filename']))
        dim=img.size
        w,h=dim[0],dim[1]
        x1=y['x1']*w
        x2=y['x2']*w
        y1=y['y1']*h
        y2=y['y2']*h
        area=(x1,y1,x2,y2)
        print(count)
        cropped_img = img.crop(area)
        cropped_img=cropped_img.convert('RGB')
        cropped_img.save("/Users/mehulgupta/Desktop/im/"+str(x)+'.jpg')
        count+=1
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
